In [1]:
import os
import cv2
import keras
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
symbols = ['e', 'u', 'y']
classes = []
data = []

for symbol in symbols:
    for image in os.listdir(symbol):
        array = cv2.imread(symbol + '/' + image, 0)
        data.append(array)
        if symbol == 'e':
            classes.append(0)
        elif symbol == 'u':
            classes.append(1)
        elif symbol == 'y':
            classes.append(2)

arr_classes = np.array(classes)
arr_data = np.array(data)

In [3]:
num_train = 480 
num_test = 120
num_classes = 3
height, width, depth = 32, 32, 1

In [4]:
x_train, x_test, y_train, y_test = train_test_split(arr_data, arr_classes, test_size = 0.2)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], depth, height, width)
    x_test = x_test.reshape(x_test.shape[0], depth, height, width)
    input_shape = (depth, height, width)
else:
    x_train = x_train.reshape(x_train.shape[0], height, width, depth)
    x_test = x_test.reshape(x_test.shape[0], height, width, depth)
    input_shape = (height, width, depth)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

('x_train shape:', (480, 32, 32, 1))
(480, 'train samples')
(120, 'test samples')


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

W0826 12:44:34.110824 140011265042240 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 12:44:34.127569 140011265042240 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 12:44:34.130172 140011265042240 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 12:44:34.155004 140011265042240 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0826 12:44:34.157677 140011265042240 deprecation_wrapp

In [7]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

W0826 12:44:42.207598 140011265042240 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 12:44:42.214214 140011265042240 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [8]:
model.fit(x_train, y_train,
          batch_size=32, epochs=50,
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Loss:', score[0])
print('Accuracy:', score[1])

W0826 12:44:47.196475 140011265042240 deprecation.py:323] From /home/ezrafell/.local/lib/python2.7/site-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 480 samples, validate on 120 samples
Epoch 1/50
480/480 [==============================] - 1s 3ms/step - loss: 1.1344 - acc: 0.4813 - val_loss: 0.8106 - val_acc: 0.8083
Epoch 2/50
480/480 [==============================] - 1s 2ms/step - loss: 0.5415 - acc: 0.8250 - val_loss: 0.2276 - val_acc: 0.9583
Epoch 3/50
480/480 [==============================] - 1s 2ms/step - loss: 0.1967 - acc: 0.9437 - val_loss: 0.0830 - val_acc: 0.9750
Epoch 4/50
480/480 [==============================] - 1s 2ms/step - loss: 0.0983 - acc: 0.9729 - val_loss: 0.0445 - val_acc: 0.9917
Epoch 5/50
480/480 [==============================] - 1s 2ms/step - loss: 0.0799 - acc: 0.9771 - val_loss: 0.0363 - val_acc: 0.9917
Epoch 6/50
480/480 [==============================] - 1s 2ms/step - loss: 0.0427 - acc: 0.9917 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 7/50
480/480 [==============================] - 1s 2ms/step - loss: 0.0252 - acc: 0.9979 - val_loss: 0.0098 - val_acc: 1.0000
Epoch 8/50
480/480 [==========